In [5]:
import numpy
import scipy.special
import scipy.misc
import matplotlib.pyplot
import imageio

%matplotlib inline

## класс сети
class Net:

    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        
        # устанавливаем количество нод во входном, скрытом и выходном слоях
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # устанавливаем learning rate
        self.lr = learningrate
        
        # генерируем начальную матрицу нужного размера, веса случайные с нормальным распределением
        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        
        # задаем функцию активации (в д.с. сигмоида)
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass
    
    def train(self, inputs_list, targets_list):
        
        # трансформируем inputs и targets в матрицы
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # inputs для скрытого слоя
        hidden_inputs = numpy.dot(self.wih, inputs)
        
        # outputs для скрытого слоя 
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # inputs для выходного слоя
        final_inputs = numpy.dot(self.who, hidden_outputs)
        
        # outputs для выходного слоя
        final_outputs = self.activation_function(final_inputs)
        
        # ошибки на выходном слое
        output_errors = targets - final_outputs
        
        # ошибки скрытого слоя
        hidden_errors = numpy.dot(self.who.T, output_errors)
        
        # обновляем веса скрытый -> выходной
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # обновляем веса входной -> скрытый
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass
    
    def query(self, inputs_list):
        
        # трансформируем inputs в матрицу
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # inputs скрытого слоя
        hidden_inputs = numpy.dot(self.wih, inputs)
        
        # outputs скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # inputs выходного слоя
        final_inputs = numpy.dot(self.who, hidden_outputs)
        
        # outputs выходного слоя
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs



In [6]:
# Задаем начальные значения. 784 = 28х28 -- формат датасета.
# Количество скрытых нод, эпох коэффициент обучения подбираем экперементально.
# На выходе получаем прогноз по цифрам от 1 до 10
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

learning_rate = 0.12

epochs = 3


n = Net(input_nodes, hidden_nodes, output_nodes, learning_rate)

training_data_file = open("mnist_train.csv", "r")
training_data_list = training_data_file.readlines()
training_data_file.close()

# тренировка

for i in range(epochs):

    for record in training_data_list:
        all_values = record.split(',')
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01

        targets = numpy.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99

        n.train (inputs, targets)
        pass
    pass

# результаты
scorecard = []

# тестирование сети
test_data_file = open("mnist_test.csv", "r")
test_data_list = test_data_file.readlines()
test_data_file.close()

for record in test_data_list:
    all_values = record.split(',')
    correct_label = int(all_values[0])
    
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    
    outputs = n.query(inputs)
    
    label = numpy.argmax(outputs)
    
    if (label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)
    pass


scorecard_array = numpy.asarray(scorecard)
print('Эффективность:', scorecard_array.sum() / scorecard_array.size)
# print(scorecard)


Эффективность:  0.9716
